In [1]:
import sys
sys.path.insert(1, '../../../..')

In [2]:
import optuna
from optuna.trial import TrialState
from optuna.samplers import TPESampler

import torch
import torch.optim as optim
import torch.nn as nn

import numpy as np
import sklearn
import copy

In [3]:
from methods.PINN import PINN

from solvers.PoissonSolver import PoissonSolver

from methods.DataDrivenMethods import DDMethod

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
params_solver = {'equation': 'Poisson', 'domain': [0., 1.], 'D': 1., 'nx': 101}

In [6]:
solver = PoissonSolver(params=params_solver)

In [7]:
params_PINN = {'layer_dims': None, 'activations': None, 'device': device, 'seed': 123}

In [8]:
Dmin, Dmax = 0.1, 10
nD = 1000
D_list = np.linspace(Dmin, Dmax, nD).reshape(-1, 1)
VD = np.ones((nD, solver.nx))*D_list

In [9]:
U_sols = solver.Vsolve(vect = 'D', D = VD)

In [10]:
d_train, d_val, u_train, u_val = sklearn.model_selection.train_test_split(D_list, U_sols, 
                                                                          test_size=0.2, 
                                                                          random_state=123)

In [11]:
nx = solver.nx
D_train_repeated = torch.Tensor(d_train.repeat(nx, 1)).view(-1, 1)
D_val_repeated = torch.Tensor(d_val.repeat(nx, 1)).view(-1, 1)

In [12]:
x = torch.Tensor(solver.x).view(-1, 1)
X_train = x.repeat(d_train.shape[0], 1)
X_val = x.repeat(d_val.shape[0], 1)

In [13]:
DX_train = torch.cat((D_train_repeated, X_train), dim=1)
DX_val = torch.cat((D_val_repeated, X_val), dim=1)
U_train = torch.Tensor(u_train).view(-1, 1)
U_val = torch.Tensor(u_val).view(-1, 1)

In [14]:
def get_dataset():
    return DX_train, DX_val, U_train, U_val

In [15]:
def loss_fn(x, y = 0):
    return torch.square(y - x).mean()

In [16]:
def define_model(trial, input_size, output_size):
    
    n_layers = trial.suggest_int("n_layers", 2, 8)
    layers = [input_size]
    activations = []
    
    for i in range(n_layers):
        out_features = trial.suggest_int("units_l{}".format(i), 10, 120)
        activation = trial.suggest_categorical("activation_l{}".format(i), ["tanh", "relu"])
        
        layers += [out_features]
        activations.append(activation)
        
    layers += [output_size]
    params_PINN_trial = copy.deepcopy(params_PINN)
    params_PINN_trial['layer_dims'] = layers
    params_PINN_trial['activations'] = activations
    return PINN(params={'solver':None, 'method':params_PINN_trial})

In [17]:
def objective(trial):
    # Generate the model.
    
    model = define_model(trial, 2, 1)
    # Generate the optimizers.
    lr = trial.suggest_float("lr", 1e-5, 1e-2, log=True)
    
    DX_train, DX_val, U_train, U_val = get_dataset()
    
    torch.manual_seed(params_PINN['seed'])
    
    n_epochs = 10000
    hyperparameters = {'lr': lr, 'epochs': n_epochs, 'optimizer': 'Adam'}
    
    # Training of the model.
    model.fit(hyperparameters=hyperparameters, 
              DX_train=DX_train, DX_val=DX_val,
              U_train=U_train, U_val=U_val, 
              data_ratio=0., physics_ratio=1.,
              #loss=None, w_r=100
             )
    
    val_loss = model.loss_dict['val']
    trial.report(min(val_loss), n_epochs)

    return min(val_loss)

In [18]:
sampler = TPESampler(seed=params_PINN['seed'])
study = optuna.create_study(direction="minimize", sampler=sampler)

[I 2023-08-22 18:20:31,339] A new study created in memory with name: no-name-dcdf92ec-1b3c-49e9-b9ec-f75a22f952c0


In [ ]:
study.optimize(objective, n_trials=10)

[tr : 1.8e-06, val : 2.0e-06]:  28%|█▉     | 2814/10000 [00:31<01:18, 91.08it/s]

In [14]:
pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  5.273067017697031e-06
  Params: 
    n_layers: 4
    units_l0: 17
    units_l1: 15
    units_l2: 26
    units_l3: 31
    lr: 0.00018591711878786343


In [15]:
activations = []
layer_dims = []

for key, value in trial.params.items():
    if key.split('_')[0] == 'activation':
        activations.append(value)
    elif key.split('_')[0] == 'units':
        layer_dims.append(value)
    elif key == 'optimizer':
        optimizer = value
    elif key == 'lr':
        lr = value

Use this to fill config_step_1.py

In [16]:
print(activations, '\n', layer_dims, '\n', '\n', lr)

[] 
 [17, 15, 26, 31] 
 
 0.00018591711878786343
